In [ ]:
!pip install boto3

import boto3
import time
from botocore.exceptions import ClientError

# AWS Boto3 client setup
cloudformation_client = boto3.client('cloudformation')
region = boto3.session.Session().region_name

In [ ]:
# Function to deploy CloudFormation stack
def deploy_cloudformation(template_file):
    with open(template_file, 'r') as file:
        cloudformation_template = file.read()
    
    try:
        response = cloudformation_client.create_stack(
            StackName='CustomerSupportChatbotGuardrailStack2',
            TemplateBody=cloudformation_template,
            Capabilities=['CAPABILITY_NAMED_IAM']
        )
        print("CloudFormation stack creation initiated...")
        stack_id = response['StackId']
        
        # Wait for stack creation to complete
        waiter = cloudformation_client.get_waiter('stack_create_complete')
        waiter.wait(StackName=stack_id)
        print("CloudFormation stack created successfully.")
        
        # Retrieve the outputs
        stack = cloudformation_client.describe_stacks(StackName=stack_id)
        outputs = stack['Stacks'][0]['Outputs']
        guardrail_identifier = None
        for output in outputs:
            print(f"{output['Description']}: {output['Value']}")
            if output['Description'] == 'The unique identifier of the guardrail':
                guardrail_identifier = output['Value']
        
        if guardrail_identifier is not None:
            print(f"Guardrail Identifier: {guardrail_identifier}")
        else:
            print("Guardrail Identifier not found in the stack outputs.")
        
        return outputs, guardrail_identifier
    except ClientError as e:
        print(f"Error creating CloudFormation stack: {e}")

#Deploy CloudFormation stack and get the guardrail identifier
outputs, guardrail_identifier = deploy_cloudformation(cfn_template_file)